In [ ]:
!pip install openpyxl
!pip3 install numpy
!pip3 install scipy
!pip install statsmodels
!pip3 install plotly

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory


In [ ]:
import pandas as pd
import nervaluate
import ast
from nervaluate import Evaluator
import re

import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import seaborn as sns
import matplotlib.pyplot as plt

# Contrast Analysis

This Notebook provides code to find significant differences in model performance across groups (e.g.: in our case, we used it to calculate the performance of prompts across centuries and entity category labels.).

Normality Test: Uses the Shapiro-Wilk test to assess whether the data for each metric is normally distributed within each group.
ANOVA Test: If all groups are normal, performs a one-way ANOVA to check for significant differences between group means.
Post Hoc Analysis: If ANOVA finds significant differences, performs Tukey's test for pairwise comparisons.
Non-parametric Test: If the normality assumption fails, uses the Kruskal-Wallis test to detect differences between groups.
Post Hoc for Non-parametric Data: If Kruskal-Wallis finds significant differences, performs pairwise Mann-Whitney U tests.

The Notebook creates a results file summarizing normality tests, ANOVA/Kruskal-Wallis tests, and post hoc comparisons for the dataset at hand.


In [ ]:
# This code built a dataframe with  with all the information compute in the 4 different scenarios: strict,exact,type and partial for the three metrics: precision, recall and F1 For the general case include all the tags in a sentence and for the different tags in each sentence. Make a contrst analysis grouping by kind of prompt and century for all the samples and for the samples order by categories. All the distributions are not normal.

values=['chain_of_thought_FS',
'chain_of_thought',
'few_shot',
'one_shot']

#Person, hay que sumar uno al indice de la izauierda y al de la derecha en la siguiente celda
ranges={ 'GENERAL':[11,23],
        'PERSON':[23,36],
        'LOCATION':[36,49],
        'FAUNA':[49,62],
        'FLORA':[62,75],
        'WEATHER':[75,88],
        'ORGANISATION':[88,101],
        'BIOME':[101,114],
        'MYTH':[114,127],
        'HUM_LANDFORM':[127,140],
        'NAT_LANDFORM':[140,153],
        'NAT_PHENOMENON':[153,166],
        'LAND_COVER':[166,179],
        }
tags=['PERSON', 'LOCATION','FAUNA','FLORA', 'WEATHER', 'ORGANISATION','BIOME','MYTH','HUM_LANDFORM','NAT_LANDFORM','NAT_PHENOMENON','LAND_COVER']
# Field for grouping in the contrast analysis. Prompt to analyze based on the type of prompt, Century  to analyze based on the Century :-)
prompt=['prompt','Century']

# Build a dataframe with all the information compute in the 4 different scenarios: strict,exact,type and partial for the three metrics: precision, recall and F1 For the general case include all the tags in a sentence and for the different tags in each sentence.

analyze=pd.DataFrame()
result=[]
path='/Users/salvador/GitHub/absa-travelogues/ABSA_data_ok/resultado_ok_century/'
for value in values:
    es_ok=pd.read_csv(path+f'TODO_normalizado_{valor}_english_fin_ok.csv')
    es_ok['prompt']=value
    result.append(es_ok)
    analyze = pd.concat(result, ignore_index=True)

for group in prompt:
        for key,lista in rangos.items():
                if key!='GENERAL':
                        analiza_1=analyze[analyze[analyze.columns[lista[0]]]==key]
                        lista_columnas=df.columns[lista[0]+1:lista[1]]
                else:
                        analiza_1=analyze
                        lista_columnas=df.columns[lista[0]:lista[1]]
                contrast_analysis(analiza_1,group,key,path,list_columnas=lista_columnas)

#pinta_violin(analiza,campo='Recall_strict',grupo='prompt')

In [ ]:
# This code performs contrast analysis grouped by the group variable.
# analyze is the dataframe to analyze and should contain the values of different metrics as well as the grouping variable if necessary.

def contrast_analysis(analyze, group_var, key, path, column_list):
    # group_var='prompt'
    # group_var='Century'
    # analyze.columns[12:24];
    columns = column_list
    # Filter the data for the different metrics
    # Check the normality of each group
    file = path + f'result_{key}_{group_var}'
    with open(file, "w") as file:
        count_samples = analyze.shape[0]
        # Display the result
        screen = f'Number of {key} samples are {count_samples}\n\n'
        file.write(screen)
        for value in columns:
            group_normality = analyze.groupby(group_var)[value].apply(lambda x: stats.shapiro(x)[1])
            screen = f"Normality test by group={group_var} for {value}\n"
            print(screen)
            print(group_normality)
            file.write(screen)
            file.write(str(group_normality))

            # Check if the data is normal in general and by group
            if group_normality.min() > 0.05:
                screen = "The data is normal in all groups. ANOVA can be performed."
                print(screen)
                file.write(screen)
                # Remove groups with less than 2 observations and null data
                stat, p_value = stats.f_oneway(
                    *[group[value].values for _, group in analyze.groupby(group_var)]
                )

                screen = f"ANOVA for {value}: p-value = {p_value}\n"
                print(screen)
                file.write(screen)

                if p_value < 0.05:
                    screen = "Significant results. Performing Tukey post hoc test...\n"
                    print(screen)
                    file.write(screen)
                    tukey = pairwise_tukeyhsd(endog=analyze[value], groups=analyze[group_var], alpha=0.05)
                    screen = "Post Hoc Test (Tukey):\n"
                    print(screen)
                    print(tukey.summary())
                    file.write(screen)
                    file.write(str(tukey.summary()))

                else:
                    screen = f"No significant differences between the {group_var} groups for the {value} index."
                    print(screen)
                    file.write(screen)
            else:
                screen = "The data is not normal in all groups. It is recommended to perform a non-parametric test such as Kruskal-Wallis.\n"
                print(screen)
                file.write(screen)
                stat, p_value = stats.kruskal(
                    *[group[value].values for _, group in analyze.groupby(group_var)]
                )
                screen = f"Kruskal-Wallis for {value}: p-value = {p_value}\n\n"
                print(screen)
                file.write(screen)
                medians = analyze.groupby(group_var)[value].median()
                means = analyze.groupby(group_var)[value].mean()
                screen = f'Medians={medians}\n'
                file.write(screen)
                screen = f'Means={means}\n'
                file.write(screen)
                print('Means', means)
                if p_value <= 0.05:
                    screen = "There are significant differences. Performing pairwise post hoc analysis (Mann-Whitney U)...\n"
                    print(screen)
                    file.write(screen)
                    group_names = analyze[group_var].unique()
                    for i in range(len(group_names)):
                        for j in range(i + 1, len(group_names)):
                            group1 = analyze[analyze[group_var] == group_names[i]][value]
                            group2 = analyze[analyze[group_var] == group_names[j]][value]
                            stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')

                            screen = (f"Comparison {group_names[i]} vs {group_names[j]}: p-value = {p_value}, : Stat, {stat}\n")
                            print(screen)
                            file.write(screen)

In [ ]:
def pinta_violin (analiza,campo,grupo):
    outliers = analiza[analiza[campo] > 1]
    if outliers.empty:
        print(f"There are no values greater than 1:\n")
    plt.figure(figsize=(10, 6))
    sns.violinplot(
        data=analyze,
        x=grupo,
        y=campo,
        hue=grupo,  # Asigna `Century` a `hue` para que la paleta funcione correctamente
        palette="Set2",
        linewidth=2,
        #inner="quartile",
        dodge=False,    # Evita que se separe por categorías del hue
        legend=False    # Desactiva la leyenda
    )
    #sns.violinplot(x='Century', y='PERSON_strict_F1', data=persona, palette='muted', inner='quartile')
    medianas = analiza.groupby(grupo)[campo].median()
    medias= analiza.groupby(grupo)[campo].mean()
    print('Medianas',medianas)
    print('Medias',medias)
    for i, median in enumerate(medianas):
        plt.scatter(i, median, color="red", s=50, zorder=5, label="Median" if i == 0 else "")
    for i, media in enumerate(medias):
        plt.scatter(i, media, color="blue", s=50, zorder=5, label="Media" if i == 0 else "")

    # Personalizar el gráfico
    #plt.legend(loc="upper left")
    plt.ylim(0, 1)
    plt.title('Group Distribution', fontsize=14)
    plt.xlabel(grupo, fontsize=12)
    plt.ylabel(campo, fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Mostrar el gráfico
    plt.show()